In [42]:
!pip install yfinance
!pip install psycopg2-binary
!pip install python-dotenv
!pip install SQLAlchemy 
import pandas as pd

In [43]:
import yfinance as yf # Son el historico de 5 años de la empresa harmony gold. Se renombran las columnas, se eliminan 3 y se crea una indicando el nombre de la empresa.
hmy = yf.Ticker('HMY')
hist = hmy.history(period="5y")
hist['Date']=hist.index
hist=hist.reset_index(drop=True)
hist= hist.rename(columns={"Open":"Apertura","High":"Precio_alto","Low":"Precio_bajo","Close":"Cierre","Volume":"Volumen","Dividends":"Dividendos","Stock Splits":"Stock","Date":"Dia"})
hist= hist.drop(columns=["Dividendos","Stock","Volumen"])
hist.insert(
    loc=0,
    column= "Empresa",
    value="Harmony Gold",
    allow_duplicates=False,
)
hist

,Empresa,Apertura,Precio_alto,Precio_bajo,Cierre,Dia
0,Harmony Gold,1.549260,1.626722,1.520211,1.578308,2018-06-21 00:00:00-04:00
1,Harmony Gold,1.578308,1.587991,1.549260,1.549260,2018-06-22 00:00:00-04:00
2,Harmony Gold,1.549260,1.568625,1.529894,1.558942,2018-06-25 00:00:00-04:00
3,Harmony Gold,1.520211,1.529894,1.471796,1.500845,2018-06-26 00:00:00-04:00
4,Harmony Gold,1.471796,1.520211,1.471796,1.500845,2018-06-27 00:00:00-04:00
...,...,...,...,...,...,...
1252,Harmony Gold,4.660000,4.700000,4.460000,4.480000,2023-06-13 00:00:00-04:00
1253,Harmony Gold,4.600000,4.660000,4.490000,4.530000,2023-06-14 00:00:00-04:00
1254,Harmony Gold,4.560000,4.590000,4.500000,4.560000,2023-06-15 00:00:00-04:00
1255,Harmony Gold,4.550000,4.570000,4.390000,4.510000,2023-06-16 00:00:00-04:00


In [44]:
hist.duplicated() #verifico que no alla duplicados.

0       False
1       False
2       False
3       False
4       False
        ...  
1252    False
1253    False
1254    False
1255    False
1256    False
Length: 1257, dtype: bool

In [45]:
Harmony_sin_duplicados= hist.drop_duplicates()#Elimino duplicados 
Harmony_sin_duplicados

,Empresa,Apertura,Precio_alto,Precio_bajo,Cierre,Dia
0,Harmony Gold,1.549260,1.626722,1.520211,1.578308,2018-06-21 00:00:00-04:00
1,Harmony Gold,1.578308,1.587991,1.549260,1.549260,2018-06-22 00:00:00-04:00
2,Harmony Gold,1.549260,1.568625,1.529894,1.558942,2018-06-25 00:00:00-04:00
3,Harmony Gold,1.520211,1.529894,1.471796,1.500845,2018-06-26 00:00:00-04:00
4,Harmony Gold,1.471796,1.520211,1.471796,1.500845,2018-06-27 00:00:00-04:00
...,...,...,...,...,...,...
1252,Harmony Gold,4.660000,4.700000,4.460000,4.480000,2023-06-13 00:00:00-04:00
1253,Harmony Gold,4.600000,4.660000,4.490000,4.530000,2023-06-14 00:00:00-04:00
1254,Harmony Gold,4.560000,4.590000,4.500000,4.560000,2023-06-15 00:00:00-04:00
1255,Harmony Gold,4.550000,4.570000,4.390000,4.510000,2023-06-16 00:00:00-04:00


In [46]:
Harmony_sin_duplicados.describe()

,Apertura,Precio_alto,Precio_bajo,Cierre
count,1257.000000,1257.000000,1257.000000,1257.000000
mean,3.409547,3.485148,3.331000,3.406069
std,1.180978,1.199706,1.150139,1.174347
min,1.394334,1.481479,1.384651,1.394334
25%,2.469132,2.595010,2.388402,2.510000
50%,3.447102,3.534248,3.380000,3.450000
75%,4.155914,4.225957,4.066806,4.153952
max,7.213739,7.368665,6.787693,6.874838


In [47]:
import os
import psycopg2
env = os.environ

In [59]:
try:
    conn =psycopg2.connect( host=env['AWS_REDSHIFT_HOST'],
                           dbname="data-engineer-database",
                           user= env['AWS_REDSHIFT_USER'],
                           password= env['AWS_REDSHIFT_PASSWORD'],
                           port="5439"
                          )
    print("me conecte")
except Exception as e:
    print("odio esto")
    print(e)  

me conecte


In [60]:
cursor = conn.cursor()
cursor.execute(f"""
create table if not exists {"jcapuzzi_coderhouse"}.Harmony_sin_duplicados (
    Empresa VARCHAR(30),
    Apertura decimal(20,2),
    Precio_alto decimal(20,2),
    Precio_bajo decimal(20,2),
    Cierre decimal(20,2),
    Dia date distkey
) sortkey(Dia);
""")
conn.commit()
cursor.close()
print("Table created!")

Table created!


In [61]:
cur= conn.cursor()
cur.execute ("SELECT * FROM Harmony_sin_duplicados")
results = cur.fetchall()
results

[]

In [62]:
from psycopg2.extras import execute_values
cur = conn.cursor ()
table_name="harmony_sin_duplicados"
columns=["Empresa","Apertura","precio_alto","Precio_bajo","Cierre","Dia"]
values=[tuple(x) for x in Harmony_sin_duplicados.to_numpy()]
insert_sql= f"INSERT INTO {table_name} ({','.join(columns)}) VALUES %s"
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")